# Product Sales Data Management System

**Assignment Submission**  
Author: Shivam Chaudhari 
Date: July 18, 2025


A Python project to manage, update, and maintain product sales and metadata using CSV, JSON, and TXT files, organized with a robust folder structure. Built to meet coursework requirements for file handling and product management.

## IMPORTS & SETUP


In [40]:
import os,json,csv
#   ONLY FOR CHECKING THE CD
# print("Current working directory:", os.getcwd())
# print("Files in directory:", os.listdir())

## LOAD DATA

In [41]:
def load_data():
    
    product_details={}
    product_descriptions={}
    path_cw=os.getcwd()  # Shows current working directory
    
    # Accesing csv file data since its in main folder not inside any sub-folder hence no need to specify the path
    with open('sales_data.csv',"r") as handle_sales:
        reader=csv.reader(handle_sales)
        next(reader)  # Skip the header row
        sales_data = { row[0]: row[1:] for row in reader }
    
    
    #Accsesing the .json file from folder --> product_details
    product_details_json_files=os.listdir('product_details')
    for json_file_name in product_details_json_files:
        #since only using file name will give fof erorr then we need to give abs or relative path
        file_path = os.path.join(path_cw,'product_details',json_file_name )
        
        # Ensure it's a file before opening (not a folder)
        if os.path.isfile(file_path) and json_file_name.endswith('.json'):
            
            #since file names are seperated like details_{sku_id}.json that's why i used split on '_' and slicing to remove '.json'
            prod_sku=json_file_name.split("_")[1][:-5]      
            
            with open(file_path, 'r') as prod_details:
                data = json.load(prod_details)  # use json.load() for file object
                product_details[prod_sku]=data
           
   
    #Accsesing the .txt file from folder --> product_descriptions
    product_descriptions_txt_files=os.listdir('product_descriptions')
    for txt_file_name in product_descriptions_txt_files:
        #Building path 
        file_path = os.path.join(path_cw,'product_descriptions',txt_file_name )
        #For file exist or not 
        if os.path.isfile(file_path) and txt_file_name.endswith('.txt'):
            with open(file_path, 'r') as prod_desc:
                prod_sku=txt_file_name.split("_")[1][:-4]
                descriptions=prod_desc.read()   #read the txt file using read which gives output in string datatype
                product_descriptions[prod_sku]=descriptions
      
    return sales_data, product_details, product_descriptions

sales_data, product_details, product_descriptions = load_data()
# print(sales_data, product_details, product_descriptions)

## UPDATE FUNCTIONS

In [42]:
# for updating dictionary sales_data using update function
def update_sales_data(to_update_prod,sales_data):
    sales_data.update(to_update_prod)
    return sales_data

# for updating dictionary product_details using update function
def update_product_details(to_update_prod,product_details):
    product_details.update(to_update_prod)
    return product_details

# for updating dictionary product_descriptions using update function
def update_product_description(to_update_prod,product_descriptions):
    product_descriptions.update(to_update_prod)
    return product_descriptions

# instead of repeting i just made a function for error message
def print_error(message):
    border = "#" * 100
    error_line = f" ERROR: {message.upper()}! "
    print(f"\n{border}\n{error_line.center(100, '#')}\n{border}\n")


def update( sales_data , product_details , product_descriptions):
    #   FOR UPDATING SALES DATA
    
    # taking input for SKU id
    prod_SKU=input("Enter Product SKU ID: ").strip()# using strip() to remove spaces from both sides to not cause any error

    # validating the id
    if len(prod_SKU)==13:
        sales=input("Enter the sales past 14 days sepreted by spaces").strip()
        
        # converting in int and also checking if they are whole number and not any charachter
        #i could used to validate directly through the for loop but then i would need to again transverse through to see all sales are whole numbers.
        sales=[int(day) for day in sales.split() if day.isdecimal() if int(day.strip()) >=0 ]
        if len(sales)!=14:
           print_error("INVALID DATA")
           return sales_data , product_details , product_descriptions
    else:
        print_error("INVALID SKU ID")
        return sales_data , product_details , product_descriptions #returning original dictionaries
    # Function calling for adding in parent dictionary
    result_sales = update_sales_data({prod_SKU: sales}, sales_data)
    
    #   FOR UPDATING SALES DETAILS
    details={}
    keys_details=['name', 'brand' , 'model' , 'specifications' , 'price' ,'availability' ]
    for w in keys_details:
        details[w]=input(f"{prod_SKU} {w}: ")
    
    # Function calling for adding in parent dictionary
    result_details = update_product_details({prod_SKU: details}, product_details)
    
    # FOR UPDATING DECRIPTION
    description=input(f"{prod_SKU} description: ")
    if not description:
        print_error("NO DESCRIPTION GIVEN")
        return sales_data , product_details , product_descriptions
    
    # Function calling for adding in parent dictionary
    result_desc = update_product_description({prod_SKU: description}, product_descriptions)
    
    # succes message
    print(f"SUCCESFULLY UPDATED DATA FOR PRODUCT {prod_SKU}".center(100, '#'))
    # returning in a tuple output
 
    return result_sales, result_details, result_desc


## RUN UPDATE() – optional

In [43]:
#  NOTE FOR UPDATING USE THE CALL BELOW u can uncomment it 
# sales_data , product_details , product_descriptions= update(sales_data , product_details , product_descriptions)

## SAVE DATA

In [45]:
# to upate the mainfolder
def dump_data(sales_data, product_details, product_descriptions, main_folder):
    """
    Save all data dictionaries to disk, recreating folder structure as needed:
      - sales_data.csv in main_folder
      - product_details JSON files in main_folder/product_details
      - product_descriptions TXT files in main_folder/product_descriptions
    """
    # Checking main folder exists
    os.makedirs(main_folder, exist_ok=True)
    
    # Dump sales_data into a CSV file in main_folder
    sales_file_path = os.path.join(main_folder, "sales_data.csv")
    with open(sales_file_path, 'w',newline='') as csvfile:
        header = ['Product_SKU'] + [f'Day{i+1}' for i in range(14)]#first heading
        writer = csv.writer(csvfile)
        writer.writerow(header)
        
        for sku, sales_list in sales_data.items():
            writer.writerow([sku]+sales_list)
    
    # Dump product_details as .json files in product_details subfolder by first making path for folder "product_details" and individual .json files
    details_folder = os.path.join(main_folder, "product_details")
    os.makedirs(details_folder, exist_ok=True)
    for sku, details in product_details.items():
        filename = f"details_{sku}.json"# since filname structure were like that no need use listdir()
        filepath = os.path.join(details_folder, filename)
        with open(filepath, 'w') as jsonfile:
            json.dump(details, jsonfile)
    
    # Dump product_descriptions as .txt files in product_descriptions subfolder by first making path for folder "product_descriptions" and individual .txt files
    descriptions_folder = os.path.join(main_folder, "product_descriptions")
    os.makedirs(descriptions_folder, exist_ok=True)
    for sku, description in product_descriptions.items():
        filename = f"details_{sku}.txt"
        filepath = os.path.join(descriptions_folder, filename)
        with open(filepath, 'w') as txtfile:
            txtfile.write(description)




#u can put any other location in raw string r"<file path>" and it will copy all the structure from cd to that fole path
dump_data(sales_data, product_details, product_descriptions,os.getcwd())